# cli

> CLI tool for declarative plugin management

In [ ]:
#| default_exp cli

In [ ]:
#| export
import json
import os
import subprocess
import sys
from pathlib import Path
from typing import Optional

import typer
import yaml

app = typer.Typer(help="CJM Plugin System CLI", no_args_is_help=True)

@app.callback()
def main() -> None:
    """CJM Plugin System CLI for managing isolated plugin environments."""
    pass

In [ ]:
#| export
def run_cmd(
    cmd: str,  # Shell command to execute
    shell: bool = True,  # Whether to run through shell
    check: bool = True  # Whether to raise on non-zero exit
) -> None:
    """Run a shell command and stream output."""
    print(f"Running: {cmd}")
    subprocess.run(cmd, shell=shell, check=check, executable='/bin/bash')

In [ ]:
#| export
def _generate_manifest(
    env_name: str,  # Name of the Conda environment
    package_name: str,  # Package source string (git URL or package name)
    manifest_dir: Path  # Directory to write manifest JSON files
) -> None:
    """Run introspection script inside the target env to generate manifest."""
    print(f"[{env_name}] Generating manifest...")
    
    # Robust Module Name Extraction
    if package_name.startswith("git+"):
        # Case: git+https://github.com/cj-mills/repo-name.git
        url = package_name[4:]
        repo_part = url.split('/')[-1]
        module_name = repo_part.split('.git')[0].replace('-', '_')
    elif package_name.startswith("-e "):
        path = package_name.split("-e ")[1].strip()
        module_name = Path(path).name.replace('-', '_')
    elif "/" in package_name or "\\" in package_name:
        module_name = Path(package_name).name.replace('-', '_')
    else:
        module_name = package_name.replace('-', '_')

    print(f"[{env_name}] Introspecting module: {module_name}")
    
    # Enhanced introspection script that also captures config_schema
    # The script:
    # 1. Gets metadata from get_plugin_metadata()
    # 2. Instantiates the plugin class to get config_schema
    # 3. Merges config_schema into metadata (if not already present)
    introspection_script = f'''
import json
import importlib
from {module_name}.meta import get_plugin_metadata

meta = get_plugin_metadata()

# Try to get config_schema from plugin instance if not in metadata
if "config_schema" not in meta:
    try:
        # Import plugin module and class
        plugin_module = meta.get("module", "{module_name}.plugin")
        plugin_class = meta.get("class", "")
        
        if plugin_module and plugin_class:
            mod = importlib.import_module(plugin_module)
            cls = getattr(mod, plugin_class)
            
            # Instantiate and get config schema
            instance = cls()
            if hasattr(instance, "get_config_schema"):
                meta["config_schema"] = instance.get_config_schema()
            
            # Clean up
            if hasattr(instance, "cleanup"):
                try:
                    instance.cleanup()
                except:
                    pass
    except Exception as e:
        # Config schema extraction is optional - continue without it
        pass

print(json.dumps(meta, indent=2))
'''
    
    # The introspection command
    introspection_cmd = f"conda run -n {env_name} python -c '{introspection_script}'"
    
    try:
        # Check output, capture stdout
        result_bytes = subprocess.check_output(introspection_cmd, shell=True, executable='/bin/bash')
        result_str = result_bytes.decode('utf-8').strip()
        
        # Robust JSON Parsing: 
        # Sometimes 'conda run' leaks warnings into stdout. We try to find the JSON block.
        try:
            start = result_str.find('{')
            end = result_str.rfind('}') + 1
            if start != -1 and end != 0:
                json_str = result_str[start:end]
                meta_json = json.loads(json_str)
            else:
                # Fallback to full string if brackets not found
                meta_json = json.loads(result_str)
        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to parse JSON from introspection output.")
            print(f"Raw Output:\n{result_str}")
            return

        plugin_name = meta_json.get('name', 'unknown')
        out_file = manifest_dir / f"{plugin_name}.json"
        
        # Log detected metadata
        if 'category' in meta_json:
            print(f"[{env_name}] Category: {meta_json['category']}")
        if 'interface' in meta_json:
            print(f"[{env_name}] Interface: {meta_json['interface']}")
        if 'config_schema' in meta_json:
            print(f"[{env_name}] Config schema: captured")
        
        with open(out_file, 'w') as f:
            f.write(json.dumps(meta_json, indent=2))
            
        print(f"[{env_name}] Wrote manifest to {out_file}")
        
    except subprocess.CalledProcessError as e:
        print(f"Error generating manifest for {env_name}:")
        print(e.output.decode() if e.output else str(e))
    except Exception as e:
        print(f"Unexpected error generating manifest: {e}")

In [ ]:
#| export
@app.command()
def install_all(
    config_path: str = typer.Option("plugins.yaml", "--config", help="Path to master config file"),
    force: bool = typer.Option(False, help="Force recreation of environments")
) -> None:
    """Install and register all plugins defined in plugins.yaml."""
    if not os.path.exists(config_path):
        typer.echo(f"Config file not found: {config_path}", err=True)
        raise typer.Exit(code=1)

    with open(config_path) as f:
        config = yaml.safe_load(f)

    # Setup manifest directory
    manifest_dir = Path.home() / ".cjm" / "plugins"
    manifest_dir.mkdir(parents=True, exist_ok=True)

    plugins = config.get('plugins', [])
    print(f"Found {len(plugins)} plugins to process.")

    for plugin in plugins:
        name = plugin.get('name')
        env_name = plugin.get('env_name')
        print(f"\n=== Processing {name} ({env_name}) ===")

        # 1. Check if Env Exists
        env_exists = False
        try:
            subprocess.check_call(
                f"conda env list | grep {env_name}", 
                shell=True, stdout=subprocess.DEVNULL
            )
            env_exists = True
        except subprocess.CalledProcessError:
            pass

        # 2. Create Environment
        if not env_exists or force:
            # SAFETY: If forcing, remove old one first to avoid flag issues
            if env_exists and force:
                print(f"Removing existing environment {env_name}...")
                run_cmd(f"conda env remove -n {env_name} -y")

            if 'env_file' in plugin:
                # NOTE: removed --force, added -n explicit override just in case
                # If the yaml file has a 'name' field, '-n' overrides it.
                run_cmd(f"conda env create -f {plugin['env_file']} -n {env_name}")
            elif 'python_version' in plugin:
                run_cmd(f"conda create -n {env_name} python={plugin['python_version']} -y")

        # 3. Install Dependencies (Pip)
        base_pip_cmd = f"conda run -n {env_name} pip install"
        
        if 'interface_libs' in plugin:
            libs = " ".join(plugin['interface_libs'])
            run_cmd(f"{base_pip_cmd} {libs}")

        if 'package' in plugin:
            run_cmd(f"{base_pip_cmd} {plugin['package']}")

        # 4. Generate Manifest
        pkg_source = plugin['package']
        _generate_manifest(env_name, pkg_source, manifest_dir)

    print("\n All operations complete.")

## Setup Host Environment

The `setup-host` command prepares the host application's Python environment by installing all unique interface libraries referenced in `plugins.yaml`. This is separate from `install-all` which sets up isolated plugin environments.

In [ ]:
#| export
@app.command("setup-host")
def setup_host(
    config_path: str = typer.Option("plugins.yaml", "--config", help="Path to master config file"),
    yes: bool = typer.Option(False, "--yes", "-y", help="Skip confirmation prompt")
) -> None:
    """Install interface libraries in the current Python environment."""
    if not os.path.exists(config_path):
        typer.echo(f"Config file not found: {config_path}", err=True)
        raise typer.Exit(code=1)

    with open(config_path) as f:
        config = yaml.safe_load(f)

    # Collect unique interface libraries from all plugins
    plugins = config.get('plugins', [])
    all_libs: set[str] = set()
    
    for plugin in plugins:
        interface_libs = plugin.get('interface_libs', [])
        all_libs.update(interface_libs)
    
    # Filter out cjm-plugin-system (already installed since we're running this CLI)
    all_libs = {lib for lib in all_libs 
                if 'cjm-plugin-system.git' not in lib and lib != 'cjm-plugin-system'}
    
    if not all_libs:
        typer.echo("No interface libraries found in config.")
        raise typer.Exit(code=0)

    # Display what will be installed
    typer.echo(f"Reading {config_path}...")
    typer.echo(f"Found {len(all_libs)} unique interface libraries:")
    for lib in sorted(all_libs):
        typer.echo(f"  - {lib}")
    typer.echo("")

    # Confirm with user unless --yes flag provided
    if not yes:
        confirm = typer.confirm("Install these in the current Python environment?")
        if not confirm:
            typer.echo("Aborted.")
            raise typer.Exit(code=0)

    # Install libraries using the current Python interpreter
    libs_str = " ".join(sorted(all_libs))
    run_cmd(f"{sys.executable} -m pip install {libs_str}")
    
    typer.echo("\nHost environment setup complete.")

## Estimate Disk Space

The `estimate-size` command estimates the disk space required for plugin environments before installation. It uses conda's dry-run feature for accurate conda package sizes and queries PyPI for pip package sizes.

In [ ]:
#| export
def _format_size(
    size_bytes: int  # Size in bytes
) -> str:  # Human-readable size string
    """Format bytes as human-readable string."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024:
            return f"{size_bytes:.1f} {unit}"
        size_bytes /= 1024
    return f"{size_bytes:.1f} PB"


def _get_pypi_size(
    package_spec: str  # Package name or git URL
) -> tuple[int, str]:  # (size_bytes, package_name)
    """Query PyPI for package download size."""
    import urllib.request
    import urllib.error
    
    # Extract package name from various formats
    package_name = package_spec
    
    # Handle git+https://github.com/user/repo-name.git
    if 'github.com' in package_spec and '.git' in package_spec:
        # Extract repo name and convert to package name
        repo_part = package_spec.split('/')[-1]
        package_name = repo_part.replace('.git', '').replace('-', '_')
    # Handle git+https://github.com/user/repo-name (no .git suffix)
    elif package_spec.startswith('git+'):
        repo_part = package_spec.rstrip('/').split('/')[-1]
        package_name = repo_part.replace('-', '_')
    # Handle package[extras] or package>=version
    elif '[' in package_spec:
        package_name = package_spec.split('[')[0]
    elif '>=' in package_spec:
        package_name = package_spec.split('>=')[0]
    elif '==' in package_spec:
        package_name = package_spec.split('==')[0]
    
    try:
        url = f"https://pypi.org/pypi/{package_name}/json"
        with urllib.request.urlopen(url, timeout=10) as resp:
            data = json.loads(resp.read().decode())
        
        # Get the largest file size (usually the wheel)
        urls = data.get('urls', [])
        if urls:
            max_size = max(u.get('size', 0) for u in urls)
            return max_size, package_name
    except (urllib.error.URLError, urllib.error.HTTPError, json.JSONDecodeError):
        pass
    
    return 0, package_name


def _estimate_conda_size(
    env_file: str,  # Path to environment.yml
    env_name: str  # Target environment name
) -> tuple[int, int]:  # (total_bytes, package_count)
    """Estimate conda package sizes using dry-run."""
    try:
        result = subprocess.run(
            ["conda", "env", "create", "-f", env_file, "-n", env_name, "--dry-run", "--json"],
            capture_output=True, text=True
        )
        
        if result.returncode != 0:
            return 0, 0
        
        data = json.loads(result.stdout)
        fetch_actions = data.get('actions', {}).get('FETCH', [])
        total_size = sum(pkg.get('size', 0) for pkg in fetch_actions)
        return total_size, len(fetch_actions)
        
    except (subprocess.SubprocessError, json.JSONDecodeError):
        return 0, 0


def _estimate_pip_sizes(
    packages: list[str]  # List of pip package specs
) -> tuple[int, int, list[tuple[str, int]]]:  # (total_bytes, found_count, [(name, size), ...])
    """Estimate pip package sizes from PyPI."""
    total_size = 0
    found_count = 0
    details = []
    
    for pkg in packages:
        size, name = _get_pypi_size(pkg)
        details.append((name, size))
        if size > 0:
            total_size += size
            found_count += 1
    
    return total_size, found_count, details

In [ ]:
#| export
from typing import List

@app.command("estimate-size")
def estimate_size(
    config_path: str = typer.Option("plugins.yaml", "--config", help="Path to master config file"),
    plugin_name: Optional[str] = typer.Option(None, "--plugin", "-p", help="Estimate for a single plugin"),
    verbose: bool = typer.Option(False, "--verbose", "-v", help="Show per-package breakdown")
) -> None:
    """Estimate disk space required for plugin environments."""
    if not os.path.exists(config_path):
        typer.echo(f"Config file not found: {config_path}", err=True)
        raise typer.Exit(code=1)

    with open(config_path) as f:
        config = yaml.safe_load(f)

    plugins = config.get('plugins', [])
    
    # Filter to single plugin if specified
    if plugin_name:
        plugins = [p for p in plugins if p.get('name') == plugin_name]
        if not plugins:
            typer.echo(f"Plugin not found: {plugin_name}", err=True)
            raise typer.Exit(code=1)

    typer.echo(f"=== Disk Space Estimates ===\n")
    
    total_conda = 0
    total_pip = 0
    
    for plugin in plugins:
        name = plugin.get('name', 'unknown')
        env_name = plugin.get('env_name', '')
        
        typer.echo(f"{name} (env: {env_name})")
        
        # Estimate conda packages
        conda_size = 0
        conda_count = 0
        if 'env_file' in plugin:
            typer.echo(f"  Analyzing conda environment...")
            conda_size, conda_count = _estimate_conda_size(plugin['env_file'], env_name)
            total_conda += conda_size
            
            if conda_size > 0:
                typer.echo(f"  Conda packages: {_format_size(conda_size)} ({conda_count} packages)")
            else:
                typer.echo(f"  Conda packages: Unable to estimate (env may already exist)")
        
        # Estimate pip packages
        pip_packages: List[str] = []
        if 'interface_libs' in plugin:
            pip_packages.extend(plugin['interface_libs'])
        if 'package' in plugin:
            pip_packages.append(plugin['package'])
        
        if pip_packages:
            pip_size, found_count, details = _estimate_pip_sizes(pip_packages)
            total_pip += pip_size
            
            not_found = len(pip_packages) - found_count
            size_str = _format_size(pip_size) if pip_size > 0 else "unknown"
            
            status_parts = []
            if found_count > 0:
                status_parts.append(f"{found_count} found on PyPI")
            if not_found > 0:
                status_parts.append(f"{not_found} not on PyPI")
            
            typer.echo(f"  Pip packages:   {size_str} compressed ({', '.join(status_parts)})")
            
            if verbose and details:
                for pkg_name, pkg_size in details:
                    if pkg_size > 0:
                        typer.echo(f"    - {pkg_name}: {_format_size(pkg_size)}")
                    else:
                        typer.echo(f"    - {pkg_name}: (not found)")
        
        typer.echo("")
    
    # Summary
    typer.echo("─" * 40)
    typer.echo("TOTAL ESTIMATES")
    typer.echo(f"  Conda packages: {_format_size(total_conda)}")
    typer.echo(f"  Pip packages:   {_format_size(total_pip)} (compressed)")
    typer.echo(f"  Combined:       {_format_size(total_conda + total_pip)}")
    typer.echo("")
    typer.echo("Note: Pip sizes are compressed downloads. Installed size is typically 2-5x larger.")
    typer.echo("      Conda estimates require the environment to not already exist.")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()